In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import pickle
import json
import torch

from PIL import Image
from matplotlib import patches
from pathlib import Path
from tqdm.autonotebook import tqdm
from typing import Dict, List, Tuple

from extract_features import get_all_annotations

/tmp/ipykernel_2604949/2517684002.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


In [3]:
dataset_file = 'annotations/midog_2022_test.csv'
img_dir = '/data/patho/MIDOG2/finalTest/'
figure_dir = 'figures/'
result_dir = 'results/'

In [4]:
import utils.constants as constants

# load data 
dataset = pd.read_csv(dataset_file)

# filter test samples 
test_dataset = dataset.query('split == "test"')

# create test codes
test_codes = {k: v for k, v in enumerate(test_dataset['tumortype'].unique())}

# get test samples and labels
test_samples = get_all_annotations(
    dataset=test_dataset, 
    img_dir_path=img_dir, 
    domain_col='tumortype'
    )

In [9]:
dataset.query('label == 1')

,filename,tumortype,scanner,x,y,label,split
0,001.tiff,Human Melanoma,Hamamatsu XR,1099.0,810.0,1,test
7,001.tiff,Human Melanoma,Hamamatsu XR,215.0,2716.0,1,test
13,001.tiff,Human Melanoma,Hamamatsu XR,4465.0,4921.0,1,test
17,001.tiff,Human Melanoma,Hamamatsu XR,1427.0,2805.0,1,test
18,001.tiff,Human Melanoma,Hamamatsu XR,5621.0,4219.0,1,test
...,...,...,...,...,...,...,...
10280,100.tiff,Feline Lymphoma,3DHISTECH,1198.5,4519.5,1,test
10287,100.tiff,Feline Lymphoma,3DHISTECH,76.5,2705.5,1,test
10291,100.tiff,Feline Lymphoma,3DHISTECH,2875.5,1202.0,1,test
10295,100.tiff,Feline Lymphoma,3DHISTECH,1465.0,538.0,1,test


In [5]:
test_codes

{0: 'Human Melanoma',
 1: 'Human Astrocytoma',
 2: 'Human Bladder Cancer',
 3: 'Canine Mammary Carcinoma',
 4: 'Canine Cutanious Mast Cell Tumor',
 5: 'Human Meningioma',
 6: 'Human Colon Carcinoma',
 7: 'Canine Hemangiosarcom',
 8: 'Feline Soft Tissue Carcinoma',
 9: 'Feline Lymphoma'}

In [6]:
def load_metrics(filename, metric, test_codes):
    test_codes_wo = {k: ''.join(v.split(' ')) for k, v in test_codes.items()}
    inverted_test_codes_wo = {v: k for k, v in test_codes_wo.items()}
    all_metric_data = json.load(open(filename, 'rb'))
    aggregates = all_metric_data['aggregates']
    metric_dict = {}
    for key in aggregates:
        if key.startswith('tumor') and key.endswith(metric):
            _, tumor, _ = key.split('_')
            metric_dict[inverted_test_codes_wo[tumor]] = aggregates[key]
    return metric_dict, all_metric_data

In [13]:
import utils.constants as constants
import plotly.express as px
import plotly.graph_objects as go 
import itertools

from plotly.subplots import make_subplots


def get_similarity(
        model_name: str, 
        result_dir: str, 
        metric_dir: str, 
        detector: str='FCOS', 
        metric: str='AP', 
        all_models: bool=False,
        num_models: int = 5,
        plot: bool=False
        ) -> None:
    
        # get dataset codes
        abbrevs = constants.MIDOG_ABBREVATIONS

        # get layer codes
        if detector == 'FCOS':
            layer_codes = constants.FCOS_LAYER_CODES
        elif detector == 'yolov7_d6':
            layer_codes = constants.YOLO_D6_LAYER_CODES
        elif detector == 'yolov7':
             layer_codes = constants.YOLO_LAYER_CODES
        else:
             raise NotImplementedError()


        if all_models:
                dfs = []
                for i in range(num_models):
                        
                    try:
                        model_name = model_name[:-1] + str(i)
                        # load hdv results
                        hdv_filename = os.path.join(result_dir, 'hdv_' + model_name + '.pkl')
                        hdv_scores = pickle.load(open(hdv_filename, 'rb'))

                        # load metric results
                        metrics_filename = os.path.join(metric_dir, model_name + '.json')
                        metric_data, _ = load_metrics(metrics_filename, metric=metric, test_codes=test_codes)

                        # create long dataframe
                        df = pd.DataFrame(hdv_scores)
                        df['Tumortype'] = df.index.map(test_codes)
                        df['Tumortype'] = df.index.map(abbrevs)
                        df[metric] = df.index.map(metric_data)
                        df = df.melt(id_vars=['Tumortype', metric], var_name='Layer' )
                        df['run'] = i
                        dfs.append(df)
                        df = pd.concat(dfs)
                    except:
                        continue

        else:

                # load hdv results
                hdv_filename = os.path.join(result_dir, 'hdv_' + model_name + '.pkl')
                hdv_scores = pickle.load(open(hdv_filename, 'rb'))

                # load metric results
                metrics_filename = os.path.join(metric_dir, model_name + '.json')
                metric_data, _ = load_metrics(metrics_filename, metric=metric, test_codes=test_codes)

                # create long dataframe
                df = pd.DataFrame(hdv_scores)
                df['Tumortype'] = df.index.map(test_codes)
                df['Tumortype'] = df.index.map(abbrevs)
                df[metric] = df.index.map(metric_data)
                df = df.melt(id_vars=['Tumortype', metric], var_name='Layer' )

        # rename layers
        df['Layer'] = df['Layer'].map(layer_codes)

        # drop layers
        df.drop(df[df['Tumortype'] == 'HAC'].index, inplace=True)
        df.drop(df[df['Layer'] == 'P1'].index, inplace=True)

        if plot:
            fig = px.scatter(
            df, x=metric, y='value', 
            color='Tumortype', facet_col='Layer', facet_col_wrap=5, labels={'value': 'HDV'})
            fig.show()

        else:
            return df 


### Compare results from all models 

In [61]:

metric_dir = 'results/'

model_names = [
    'yolov7_d6_ALL_0', 
    'yolov7_all_0',
    # 'LitFCOS_all_R50_bt_0', 
    # 'LitFCOS_all_R50_imagen_fda_0', 
    # 'LitFCOS_all_R50_mocov2_fda_0', 
    # 'LitFCOS_all_AdamExpLR_final_0', 
    # 'LitFCOS_HBC_AdamExpLR_final_0',
    # 'LitFCOS_HBC_R50_bt_freeze_0',
    # 'LitFCOS_HBC_R50_bt_freeze_fda_0',
    # 'LitFCOS_HBC_R50_imagen_0'
    ]

model_codes = {
    # 'LitFCOS_all_R50_bt': 'FCOS50_all_bt',
    # 'LitFCOS_all_R50_imagen_fda': 'FCOS50_all_imagen_fda',
    # 'LitFCOS_all_R50_mocov2_fda': 'FCOS50_all_mocov2_fda',
    # 'LitFCOS_all_AdamExpLR_final': 'FCOS50_all_imagen',
    # 'LitFCOS_HBC_AdamExpLR_final': 'FCOS50_hbc_imagen',
    'yolov7_d6_ALL': 'Yolov7_D6_all_coco',
    'yolov7_all': 'Yolov7_all_coco',
    # 'LitFCOS_HBC_R50_bt_freeze': 'FCOS50_hbc_bt_freeze',
    # 'LitFCOS_HBC_R50_bt_freeze_fda': 'FCOS50_hbc_bt_freeze_fda',
    # 'LitFCOS_HBC_R50_imagen': 'FCOS50_hbc_imagen_updated'
}

metric = 'AP'

dfs = []

for model_name in model_names:

    if 'yolov7_d6' in model_name:
        detector = 'yolov7_d6'
    elif 'yolov7' in model_name:
        detector = 'yolov7'
    else:
        detector = 'FCOS'

    model_df = get_similarity(model_name, result_dir='./results/reduced_dims100', detector=detector, metric=metric, metric_dir=metric_dir, all_models=True)
    model_df['model_name'] = model_codes[model_name[:-2]]

    dfs.append(model_df)


df = pd.concat(dfs)

In [62]:
# group by 'model_name' and 'Layer', calculate mean, std, and count
agg_df = df.groupby('model_name')[metric].agg(['mean', 'std', 'count']).reset_index()

# calculate the 90% confidence interval
confidence_level = 0.90
z_score = 1.645  # approximate value for 90% CI in a normal distribution

# calculate the margin of error
agg_df['margin_of_error'] = z_score * (agg_df['std'] / np.sqrt(agg_df['count']))

# calculate the lower and upper bounds of the CI
agg_df['lower_ci'] = agg_df['mean'] - agg_df['margin_of_error']
agg_df['upper_ci'] = agg_df['mean'] + agg_df['margin_of_error']

# format the mean and confidence interval into a single string
agg_df[metric] = agg_df.apply(lambda row: f"{row['mean']:.4f} ({row['lower_ci']:.4f}, {row['upper_ci']:.4f})", axis=1)

agg_df[['model_name', metric]]

,model_name,AP
0,Yolov7_D6_all_coco,"0.6488 (0.6442, 0.6534)"
1,Yolov7_all_coco,"0.4543 (0.4469, 0.4617)"


In [63]:
# group by 'model_name' and 'Layer', calculate mean, std, and count
agg_df = df.groupby(['model_name', 'Layer'])['value'].agg(['mean', 'std', 'count']).reset_index()

# calculate the 90% confidence interval
confidence_level = 0.90
z_score = 1.645  # approximate value for 90% CI in a normal distribution

# calculate the margin of error
agg_df['margin_of_error'] = z_score * (agg_df['std'] / np.sqrt(agg_df['count']))

# calculate the lower and upper bounds of the CI
agg_df['lower_ci'] = agg_df['mean'] - agg_df['margin_of_error']
agg_df['upper_ci'] = agg_df['mean'] + agg_df['margin_of_error']

# format the mean and confidence interval into a single string
agg_df['mean_ci'] = agg_df.apply(lambda row: f"{row['mean']:.2f} ({row['lower_ci']:.2f}, {row['upper_ci']:.2f})", axis=1)

# pivot the DataFrame to make it wide format with the formatted mean (CI)
pivot_df = agg_df.pivot(index='model_name', columns='Layer', values='mean_ci')

pivot_df.reset_index(inplace=True)

In [64]:
cols = ['model_name', 'C2', 'C3', 'C4', 'C5', 'P3', 'P4', 'O3', 'O4', 'O5']
pivot_df[cols]

Layer,model_name,C2,C3,C4,C5,P3,P4,O3,O4,O5
0,Yolov7_D6_all_coco,"0.16 (0.15, 0.17)","0.38 (0.36, 0.39)","0.49 (0.48, 0.51)","0.36 (0.33, 0.39)","0.46 (0.44, 0.48)","0.48 (0.45, 0.51)","0.60 (0.60, 0.61)","0.54 (0.52, 0.55)","0.32 (0.29, 0.34)"
1,Yolov7_all_coco,"0.19 (0.18, 0.20)","0.30 (0.28, 0.31)","0.15 (0.12, 0.18)","0.63 (0.50, 0.77)","0.26 (0.25, 0.28)","0.27 (0.25, 0.29)","0.63 (0.63, 0.64)","0.40 (0.39, 0.42)","0.33 (0.31, 0.35)"


In [65]:
detection = ['O3', 'O4', 'O5']
agg_df.query("Layer in @detection").groupby('model_name').aggregate({'mean': 'mean'})

,mean
model_name,
Yolov7_D6_all_coco,0.486024
Yolov7_all_coco,0.455266


In [67]:
agg_df.groupby('model_name').aggregate({'mean': 'mean'})

,mean
model_name,
Yolov7_D6_all_coco,0.333086
Yolov7_all_coco,0.294530
